In [ ]:
! pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.0 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
dataset = load_dataset("roneneldan/TinyStories")

Repo card metadata block was not found. Setting CardData to empty.


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [ ]:
import tqdm

In [ ]:
# End of Word Token
end_word_token = "|"
accepted_chars =  [' ', '!', '"', "'", ',', '.', '\n', '?']

In [ ]:
def filter_chars(string):
  return ''.join([char for char in string if char in accepted_chars])

def filter_alphabet(string):
  return ''.join([char for char in string if char.isalpha() or char == ' ' or char == '\'' or char == '\n'])

In [ ]:
# Load in and get words in training data
data_sz = len(dataset['train'])

words = {}

with tqdm.tqdm(total = data_sz) as t:
  for i in range(data_sz):
    example = filter_alphabet(dataset['train'][i]['text'])
    for word in example.split():
      if len(word) > 25:
        continue
      word += end_word_token
      if word not in words:
        words[word] = 0
      words[word] += 1
    t.update(1)

100%|██████████| 2119719/2119719 [08:04<00:00, 4378.90it/s]


In [ ]:
# Convert dictionary to a list of tuples so it can store lists as keys
words = [([char for char in key], value) for key, value in words.items()]

In [ ]:
bpe_iterations = 10000

# Create and train custom BPE tokenizer
def bpe_cycle():
  pair_counts = {}

  for word, freq in words:
    for i in range(len(word) - 1):
      if word[i] + word[i + 1] not in pair_counts:
        pair_counts[word[i] + word[i + 1]] = 0
      pair_counts[word[i] + word[i + 1]] += freq

  max_pair = list(pair_counts.keys())[0]
  for i in pair_counts.keys():
    if pair_counts[i] > pair_counts[max_pair]:
      max_pair = i

  print(f"Generated Token: {max_pair}")

  for word, _ in words:
    i = 0

    while i < len(word) - 1:
      if word[i] + word[i + 1] == max_pair:
        word[i] = word[i] + word[i + 1]
        del word[i + 1]
      i += 1

for _ in range(bpe_iterations):
  print(f"BPE Cycle {_}: ", end = '')
  bpe_cycle()

Streaming output truncated to the last 5000 lines.
BPE Cycle 5001: Generated Token: mome
BPE Cycle 5002: Generated Token: worse|
BPE Cycle 5003: Generated Token: parts|
BPE Cycle 5004: Generated Token: thering|
BPE Cycle 5005: Generated Token: universe|
BPE Cycle 5006: Generated Token: awe|
BPE Cycle 5007: Generated Token: Jo|
BPE Cycle 5008: Generated Token: vacation|
BPE Cycle 5009: Generated Token: panic|
BPE Cycle 5010: Generated Token: grin
BPE Cycle 5011: Generated Token: rin
BPE Cycle 5012: Generated Token: remained|
BPE Cycle 5013: Generated Token: examine|
BPE Cycle 5014: Generated Token: thermome
BPE Cycle 5015: Generated Token: ented|
BPE Cycle 5016: Generated Token: shooting|
BPE Cycle 5017: Generated Token: brain|
BPE Cycle 5018: Generated Token: anges|
BPE Cycle 5019: Generated Token: folded|
BPE Cycle 5020: Generated Token: sparkles|
BPE Cycle 5021: Generated Token: hurricane|
BPE Cycle 5022: Generated Token: floated|
BPE Cycle 5023: Generated Token: neighborhood|
BPE Cy

In [ ]:
ind_to_char = []
char_to_ind = {}

for char in accepted_chars:
  ind_to_char.append(char)
  char_to_ind[char] = len(ind_to_char) - 1

for word, _ in words:
  for token in word:
    if token not in char_to_ind.keys():
      ind_to_char.append(token)
      char_to_ind[token] = len(ind_to_char) - 1

# Save the tokenizer to a file so we can reuse it
with open("/content/ind_to_char.txt", "w") as f:
  f.write(str(ind_to_char))
with open("/content/char_to_ind.txt", "w") as f:
  f.write(str(char_to_ind))